In [192]:
import numpy as np
import matplotlib.pyplot as plt

## TicTacToe

In [193]:
from copy import deepcopy

class Board():
    def __init__(self, board=None):
        self.player_1 = 'x'
        self.player_2 = 'o'
        self.empty_square = '.'
        
        self.position = {}
        self.init_board()
        
        if board is not None:
            self.__dict__ = deepcopy(board.__dict__)
    
    def init_board(self):
        for row in range(3):
            for col in range(3):
                self.position[row, col] = self.empty_square
    
    def make_move(self, row, col):
        board = Board(self)
        board.position[row, col] = self.player_1
        (board.player_1, board.player_2) = (board.player_2, board.player_1)
    
        return board
    
    def is_draw(self):
        for row, col in self.position:
            if self.position[row, col] == self.empty_square:
                return False
        return True
    
    def is_win(self):
        for col in range(3):
            winning_sequence = []
            
            for row in range(3):
                if self.position[row, col] == self.player_2:
                    winning_sequence.append((row, col)) 
                if len(winning_sequence) == 3:
                    return True
        
        for row in range(3):
            winning_sequence = []
            
            for col in range(3):
                if self.position[row, col] == self.player_2:
                    winning_sequence.append((row, col)) 
                if len(winning_sequence) == 3:
                    return True
    
        winning_sequence = []
        for row in range(3):
            col = row
        
            if self.position[row, col] == self.player_2:
                winning_sequence.append((row, col))
            if len(winning_sequence) == 3:
                return True
        
        winning_sequence = []
        for row in range(3):
            col = 3 - row - 1
        
            if self.position[row, col] == self.player_2:
                winning_sequence.append((row, col))   
            if len(winning_sequence) == 3:
                return True
        return False
    
    def generate_states(self):
        actions = []
        
        for row in range(3):
            for col in range(3):
                if self.position[row, col] == self.empty_square:
                    actions.append(self.make_move(row, col))
        return actions
    
    def __str__(self):
        return self.position

## Monte Carlo Tree Search

In [194]:
import math
import random

class TreeNode():
    def __init__(self, board, parent):
        self.board = board
        
        if self.board.is_win() or self.board.is_draw():
            self.is_terminal = True   
        else:
            self.is_terminal = False
        
        self.is_fully_expanded = self.is_terminal
        self.parent = parent
        self.visits = 0
        self.score = 0
        self.children = {}

class MCTS():
    def search(self, initial_state):
        self.root = TreeNode(initial_state, None)

        for iter in range(1000):
            node = self.select(self.root)
            score = self.rollout(node.board)
            self.backpropagate(node, score) 
        try:
            return self.get_best_move(self.root, 0)
        
        except:
            pass
    
    def select(self, node):
        while not node.is_terminal:
            if node.is_fully_expanded:
                node = self.get_best_move(node, 2)
            else:
                return self.expand(node)
       
        return node
    
    def expand(self, node):
        states = node.board.generate_states()
        
        for state in states:
            if str(state.position) not in node.children:
                new_node = TreeNode(state, node)
                node.children[str(state.position)] = new_node
                
                if len(states) == len(node.children):
                    node.is_fully_expanded = True
                
                return new_node
    
    def rollout(self, board):
        while not board.is_win():
            try:
                board = random.choice(board.generate_states())
            except:
                return 0
        
        if board.player_2 == 'x': return 1
        elif board.player_2 == 'o': return -1

    def backpropagate(self, node, score):
        while node is not None:
            node.visits += 1
            node.score += score
            node = node.parent
    
    def get_best_move(self, node, exploration_constant):
        best_score = float('-inf')
        best_moves = []
        
        for child_node in node.children.values():
            if child_node.board.player_2 == 'x': current_player = 1
            elif child_node.board.player_2 == 'o': current_player = -1
            
            move_score = current_player * child_node.score / child_node.visits + exploration_constant * math.sqrt(math.log(node.visits / child_node.visits))                                        

            if move_score > best_score:
                best_score = move_score
                best_moves = [child_node]
            
            elif move_score == best_score:
                best_moves.append(child_node)
            
        return random.choice(best_moves)

In [201]:
%matplotlib qt
board = Board()
mcts = MCTS()
def addmove(x, y):
    global board
    if board.position[y, x] == board.empty_square:
        t = np.linspace(0,2*np.pi, 30)
        if board.player_1 == 'o':
            ax.plot(x*100+50+45*np.cos(t), y*100+50+45*np.sin(t), 'r', linewidth=8)
        else:
            ax.plot([x*100+5, x*100+95], [y*100+5, y*100+95], 'g', linewidth=8)
            ax.plot([x*100+95, x*100+5], [y*100+5, y*100+95], 'g', linewidth=8)
        fig.canvas.draw()
        board = board.make_move(y, x)

        if board.is_win():
            print('player "%s" has won the game!\n' % board.player_2)
        elif board.is_draw():
            print('Game is drawn!\n')
    else:
        print(' Illegal move!')

def automove():
    global board
    dict1 = board.__str__()
    ai_board = mcts.search(board).board
    dict2 = ai_board.__str__()
    pos = None
    for i,((a,b),(c,d)) in enumerate(zip(dict1.items(), dict2.items())):
        if b == '.' and d != '.':
            pos = c
    addmove(pos[1], pos[0])

def click_handler(event):
    x, y = np.int32([event.xdata/100, event.ydata/100]).clip(0,2)
    addmove(x, y)
    try:
        automove()
    except:
        pass

fig, ax = plt.subplots()
ax.axis([0, 300, 0, 300])     
ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
ax.xaxis.tick_top()                     # and move the X-Axis      
ax.yaxis.tick_left()                    # remove right y-Ticks

for i in [100,200]:
    plt.plot([i,i], [0,300], 'k')
    plt.plot([0,300], [i,i], 'k')
fig.canvas.mpl_connect('button_press_event', click_handler)

14

Game is drawn!

